In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05')
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')
spy = yf.download('SPY', start='2019-01-01', end='2025-03-05')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
def calculate_vortex(df, value, n=14):
    high = df[("High", value)]
    low = df[("Low", value)]
    close = df[("Close", value)]

    # Calculate VM+ and VM-
    vm_plus = abs(high - low.shift(1))   # |Today's High - Yesterday's Low|
    vm_minus = abs(low - high.shift(1))  # |Today's Low - Yesterday's High|

    # Calculate True Range (TR)
    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    # Rolling sum for lookback period
    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    # Compute VI+ and VI-
    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    return vi_plus, vi_minus

In [4]:
tsla['VI+'], tsla['VI-'] = calculate_vortex(tsla, 'TSLA')
xly['VI+'], xly['VI-'] = calculate_vortex(xly, 'XLY')
spy['VI+'], spy['VI-'] = calculate_vortex(spy, 'SPY')

In [24]:
import requests
import pandas as pd

url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&date_from=20250101T0130&date_to=20250301T0130&limit=1000&tickers=XLY&apikey=PNM5EHRALIOT1CKJ'

response = requests.get(url)

if response.status_code == 200:
    sentiment_data = response.json()
    sentiment_df = pd.DataFrame(sentiment_data['feed']) 
    print(sentiment_df.head())
else:
    print("API call failed:", response.status_code)

sentiment_json = response.json()
print(sentiment_json)

KeyError: 'feed'

In [6]:
sentiment_data = []
for news_item in sentiment_json.get("feed", []):
    sentiment_data.append({
            "time_published": pd.to_datetime(news_item["time_published"]),
            "sentiment_score": news_item["overall_sentiment_score"],
            "sentiment_label": news_item["overall_sentiment_label"],
    })
sentiment_data = pd.DataFrame(sentiment_data)

In [7]:
sentiment_data['time_published'] = sentiment_data['time_published'].dt.date
sentiment_scores_filtered = sentiment_data[pd.to_datetime(sentiment_data['time_published']).isin(tsla.index)]
sentiment_scores_filtered = sentiment_scores_filtered.groupby('time_published')['sentiment_score'].mean().reset_index()

In [8]:
# Fix the multi-level column issue by selecting the 'Volume' column and resetting its name
xly_volume = xly[('Volume', 'XLY')].rename('Volume')

# Ensure the index of tsla_volume is a column and convert it to match the type of time_published
xly_volume = xly_volume.reset_index()
xly_volume['Date'] = pd.to_datetime(xly_volume['Date'])

# Convert time_published to datetime
sentiment_scores_filtered['time_published'] = pd.to_datetime(sentiment_scores_filtered['time_published'])
# Merge the dataframes
merged_data = pd.merge(xly_volume, sentiment_scores_filtered, left_on='Date', right_on='time_published', how='inner')
merged_data['Weighted_Sentiment'] = merged_data['Volume'] * merged_data['sentiment_score']
merged_data['5_day_avg_sentiment'] = merged_data['Weighted_Sentiment'].rolling(window=5).mean()
merged_data['Buy_Condition'] = merged_data['5_day_avg_sentiment'] > 0
merged_data['5_day_avg_sentiment_norm'] = merged_data['5_day_avg_sentiment']/merged_data['Volume'].mean()

In [9]:
merged_data.head()

,Date,Volume,time_published,sentiment_score,Weighted_Sentiment,5_day_avg_sentiment,Buy_Condition,5_day_avg_sentiment_norm
0,2025-02-04,1993500,2025-02-04,0.239953,478346.305500,NaN,False,NaN
1,2025-02-05,2951400,2025-02-05,0.222392,656368.486650,NaN,False,NaN
2,2025-02-06,3022900,2025-02-06,0.245102,740918.047217,NaN,False,NaN
3,2025-02-07,2768200,2025-02-07,0.251546,696328.497353,NaN,False,NaN
4,2025-02-10,2981900,2025-02-10,0.246183,734092.874707,661210.842285,True,0.195118


## volatility

In [10]:

# Flatten MultiIndex columns 
xly.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col
    for col in xly.columns
]

# Calculate True Range
xly["prev_close"] = xly["Close_XLY"].shift(1)
xly["tr1"] = xly["High_XLY"] - xly["Low_XLY"]
xly["tr2"] = abs(xly["High_XLY"] - xly["prev_close"])
xly["tr3"] = abs(xly["Low_XLY"] - xly["prev_close"])

xly["true_range"] = xly[["tr1", "tr2", "tr3"]].max(axis=1)

# 10-day ATR
xly["ATR_10"] = xly["true_range"].rolling(window=10).mean()

# ---- STEP 4: Calculate ATR as a percentage of closing price ----
xly["atr_pct"] = xly["ATR_10"] / xly["Close_XLY"]

# allocating the capital

def position_size(row):
    if row["atr_pct"] < 0.03:  # < 3% volatility → low risk
        return 0.01  # allocate 1% of capital
    else:  # ≥ 3% volatility → high risk
        return 0.005  # allocate 0.5% of capital

xly["position_size"] = xly.apply(position_size, axis=1)

# ---- STEP 6: Optional - Capital allocation per trade ----
#capital = 100000 # Example: $100K total portfolio
#xly["allocation_dollars"] = xly["position_size"] * capital

# ---- Preview ----
print(xly[["Close_XLY", "ATR_10", "atr_pct", "position_size"]].tail(10))


             Close_XLY    ATR_10   atr_pct  position_size
Date                                                     
2025-02-19  225.618988  2.870099  0.012721           0.01
2025-02-20  223.674316  2.919964  0.013055           0.01
2025-02-21  217.790527  3.453495  0.015857           0.01
2025-02-24  216.972778  3.270997  0.015076           0.01
2025-02-25  215.835892  3.511334  0.016269           0.01
2025-02-26  214.948349  3.602083  0.016758           0.01
2025-02-27  211.846878  3.751672  0.017709           0.01
2025-02-28  215.367203  3.836439  0.017813           0.01
2025-03-03  211.398117  4.429805  0.020955           0.01
2025-03-04  207.668396  4.845659  0.023334           0.01


In [11]:
import plotly.express as px
fig = px.line(xly, x=xly.index, y="atr_pct", title="ATR% Over Time")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [12]:
import plotly.express as px

# Filter only 2025 data
xly_2025 = xly[xly.index.year == 2025]

# Plot
fig = px.line(xly_2025, x=xly_2025.index, y="atr_pct", title="ATR% Over Time (2025 Only)")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [13]:
merged_data = pd.merge(merged_data, xly, on='Date', how='left')


In [14]:
# Calculate ATR percentage
merged_data['atr_pct'] = merged_data['ATR_10'] / merged_data['Close_XLY']

# Vortex crossover logic
merged_data['VI_Cross_Up'] = (merged_data['VI+_'] > merged_data['VI-_']) & (merged_data['VI+_'].shift(1) <= merged_data['VI-_'].shift(1))
merged_data['VI_Cross_Down'] = (merged_data['VI-_'] > merged_data['VI+_']) & (merged_data['VI-_'].shift(1) <= merged_data['VI+_'].shift(1))

# Initialize signal & state columns
merged_data['Buy_Signal'] = False
merged_data['Sell_Signal'] = False
merged_data['Position'] = 0
merged_data['Entry_Type'] = None  # aggressive/conservative

# Trailing stop logic variables
in_position = False
peak_price = 0

for i in range(1, len(merged_data)):
    row = merged_data.iloc[i]
    idx = merged_data.index[i]
    # Buy condition
    if not in_position or row['VI_Cross_Up'] or row['5_day_avg_sentiment_norm']>0:
        merged_data.at[idx, 'Buy_Signal'] = True
        merged_data.at[idx, 'Position'] = 1
        in_position = True
        peak_price = row['Close_XLY']

        # Entry Type: aggressive if ATR < 3%, else conservative
        if row['atr_pct'] < 0.03:
            merged_data.at[idx, 'Entry_Type'] = 'aggressive'
        else:
            merged_data.at[idx, 'Entry_Type'] = 'conservative'

    # While in position, check for trailing stop or VI cross down
    elif in_position:
        current_price = row['Close_XLY']
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03 or row['VI_Cross_Down']:
            merged_data.at[idx, 'Sell_Signal'] = True
            merged_data.at[idx, 'Position'] = 0
            in_position = False
        else:
            merged_data.at[idx, 'Position'] = 1

# Show result counts
print("Buy signals:", merged_data['Buy_Signal'].sum())
print("Sell signals:", merged_data['Sell_Signal'].sum())
print("Aggressive entries:", (merged_data['Entry_Type'] == 'aggressive').sum())
print("Conservative entries:", (merged_data['Entry_Type'] == 'conservative').sum())


Buy signals: 17
Sell signals: 1
Aggressive entries: 17
Conservative entries: 0


In [15]:
import plotly.graph_objects as go

fig = go.Figure()

# Plot merged_data closing price
fig.add_trace(go.Scatter(
    x=merged_data.index, 
    y=merged_data['Close_XLY'], 
    mode='lines', 
    name='merged_data Price', 
    line=dict(color='blue')
))

# Aggressive buys
fig.add_trace(go.Scatter(
    x=merged_data[(merged_data['Buy_Signal']) & (merged_data['Entry_Type'] == 'aggressive')].index,
    y=merged_data[(merged_data['Buy_Signal']) & (merged_data['Entry_Type'] == 'aggressive')]['Close_XLY'],
    mode='markers',
    name='Buy (Aggressive)',
    marker=dict(symbol='triangle-up', color='limegreen', size=10)
))

# Conservative buys
fig.add_trace(go.Scatter(
    x=merged_data[(merged_data['Buy_Signal']) & (merged_data['Entry_Type'] == 'conservative')].index,
    y=merged_data[(merged_data['Buy_Signal']) & (merged_data['Entry_Type'] == 'conservative')]['Close_XLY'],
    mode='markers',
    name='Buy (Conservative)',
    marker=dict(symbol='triangle-up', color='green', size=10)
))

# Sells
fig.add_trace(go.Scatter(
    x=merged_data[merged_data['Sell_Signal']].index,
    y=merged_data[merged_data['Sell_Signal']]['Close_XLY'],
    mode='markers',
    name='Sell Signal',
    marker=dict(symbol='triangle-down', color='red', size=10)
))

fig.update_layout(
    title='merged_data Buy/Sell Signals Over Time',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_white',
    height=600
)

fig.show()


In [16]:
import plotly.graph_objects as go

# Ensure 'Date' is datetime and set as index if needed
merged_data['Date'] = pd.to_datetime(merged_data['Date'])

fig = go.Figure()

# Plot 5-day Avg Sentiment
fig.add_trace(go.Scatter(
    x=merged_data['Date'],
    y=merged_data['5_day_avg_sentiment_norm'],
    mode='lines+markers',
    name='5-Day Avg Sentiment',
    line=dict(color='blue')
))

# Plot ATR %
fig.add_trace(go.Scatter(
    x=merged_data['Date'],
    y=merged_data['atr_pct'],
    mode='lines+markers',
    name='ATR %',
    yaxis='y2',
    line=dict(color='orange')
))

# Optional: Highlight Buy Signal Dates (even though there are none now)
fig.add_trace(go.Scatter(
    x=merged_data.loc[merged_data['Buy_Signal'], 'Date'],
    y=merged_data.loc[merged_data['Buy_Signal'], '5_day_avg_sentiment_norm'],
    mode='markers',
    marker=dict(color='green', size=10, symbol='star'),
    name='Buy Signal'
))

# Add dual axis layout
fig.update_layout(
    title="5-Day Sentiment vs ATR % (with Buy Signals)",
    xaxis_title='Date',
    yaxis=dict(title='5-Day Avg Sentiment'),
    yaxis2=dict(title='ATR %', overlaying='y', side='right'),
    legend=dict(x=0.01, y=0.99),
    height=500
)

fig.show()


In [17]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=merged_data.index, y=merged_data['Close_XLY'], mode='lines', name='merged_data Price'))

# Buy markers
fig.add_trace(go.Scatter(
    x=merged_data[merged_data['Buy_Signal']].index,
    y=merged_data[merged_data['Buy_Signal']]['Close_XLY'],
    mode='markers',
    marker=dict(symbol='triangle-up', size=10, color='green'),
    name='Buy Signal'
))

# Sell markers
fig.add_trace(go.Scatter(
    x=merged_data[merged_data['Sell_Signal']].index,
    y=merged_data[merged_data['Sell_Signal']]['Close_XLY'],
    mode='markers',
    marker=dict(symbol='triangle-down', size=10, color='red'),
    name='Sell Signal'
))

fig.update_layout(title='XLY Buy & Sell Signals', template='plotly_white')
fig.show()


In [18]:
xly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Close_XLY      1551 non-null   float64
 1   High_XLY       1551 non-null   float64
 2   Low_XLY        1551 non-null   float64
 3   Open_XLY       1551 non-null   float64
 4   Volume_XLY     1551 non-null   int64  
 5   VI+_           1537 non-null   float64
 6   VI-_           1537 non-null   float64
 7   prev_close     1550 non-null   float64
 8   tr1            1551 non-null   float64
 9   tr2            1550 non-null   float64
 10  tr3            1550 non-null   float64
 11  true_range     1551 non-null   float64
 12  ATR_10         1542 non-null   float64
 13  atr_pct        1542 non-null   float64
 14  position_size  1551 non-null   float64
dtypes: float64(14), int64(1)
memory usage: 193.9 KB


In [19]:
capital = 100000
in_position = False
entry_price = 0
position_value = 0
cash = capital
returns = []

for i in range(len(merged_data)):
    row = merged_data.iloc[i]
    
    # Buy
    if row['Buy_Signal'] and not in_position:
        position_size = row['position_size']
        position_value = cash * position_size
        entry_price = row['Close_XLY']
        shares_bought = position_value / entry_price
        cash -= position_value
        in_position = True
        
    # Sell
    elif row['Sell_Signal'] and in_position:
        exit_price = row['Close_XLY']
        proceeds = shares_bought * exit_price
        profit = proceeds - position_value
        cash += proceeds
        returns.append(profit)
        in_position = False
        position_value = 0
        entry_price = 0

# Final capital
final_value = cash + (shares_bought * row['Close_XLY'] if in_position else 0)
total_return = final_value - capital

print(f"Final Capital: ${final_value:.2f}")
print(f"Total Return: ${total_return:.2f}")
print(f"Total Trades: {len(returns)}")
print(f"Average Profit per Trade: ${np.mean(returns):.2f}")


Final Capital: $99904.34
Total Return: $-95.66
Total Trades: 1
Average Profit per Trade: $-15.67


## Without sentiment code

In [20]:
xly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Close_XLY      1551 non-null   float64
 1   High_XLY       1551 non-null   float64
 2   Low_XLY        1551 non-null   float64
 3   Open_XLY       1551 non-null   float64
 4   Volume_XLY     1551 non-null   int64  
 5   VI+_           1537 non-null   float64
 6   VI-_           1537 non-null   float64
 7   prev_close     1550 non-null   float64
 8   tr1            1551 non-null   float64
 9   tr2            1550 non-null   float64
 10  tr3            1550 non-null   float64
 11  true_range     1551 non-null   float64
 12  ATR_10         1542 non-null   float64
 13  atr_pct        1542 non-null   float64
 14  position_size  1551 non-null   float64
dtypes: float64(14), int64(1)
memory usage: 193.9 KB


In [21]:
# Without sentiment score
xly_copy = xly.copy()
xly_copy['atr_pct'] = xly_copy['ATR_10'] / xly_copy['Close_XLY']

# Create Buy Signal (assuming VI_Cross_Up is defined elsewhere)
xly_copy['Buy_Signal'] = xly_copy['VI+_'] > xly_copy['VI-_']  # Vortex crossover
# + add any other buy conditions here...

# Create Sell Signal (basic)
xly_copy['Sell_Signal'] = xly_copy['VI-_'] > xly_copy['VI+_']

# Initialize position state
xly_copy['Position'] = 0
peak_price = 0

for i in range(1, len(xly_copy)):
    if xly_copy['Buy_Signal'].iloc[i]:
        xly_copy.at[xly_copy.index[i], 'Position'] = 1
        peak_price = xly_copy['Close_XLY'].iloc[i]
    elif xly_copy['Position'].iloc[i - 1] == 1:
        current_price = xly_copy['Close_XLY'].iloc[i]
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03:
            xly_copy.at[xly_copy.index[i], 'Sell_Signal'] = True  # trailing stop
            xly_copy.at[xly_copy.index[i], 'Position'] = 0
        else:
            xly_copy.at[xly_copy.index[i], 'Position'] = 1


In [22]:
capital = 100000
in_position = False
entry_price = 0
position_value = 0
cash = capital
returns = []

for i in range(len(xly_copy)):
    row = xly_copy.iloc[i]
    
    # Buy
    if row['Buy_Signal'] and not in_position:
        position_size = row['position_size']
        position_value = cash * position_size
        entry_price = row['Close_XLY']
        shares_bought = position_value / entry_price
        cash -= position_value
        in_position = True
        
    # Sell
    elif row['Sell_Signal'] and in_position:
        exit_price = row['Close_XLY']
        proceeds = shares_bought * exit_price
        profit = proceeds - position_value
        cash += proceeds
        returns.append(profit)
        in_position = False
        position_value = 0
        entry_price = 0

# Final capital
final_value = cash + (shares_bought * row['Close_XLY'] if in_position else 0)
total_return = final_value - capital

print(f"Final Capital: ${final_value:.2f}")
print(f"Total Return: ${total_return:.2f}")
print(f"Total Trades: {len(returns)}")
print(f"Average Profit per Trade: ${np.mean(returns):.2f}")


Final Capital: $100732.95
Total Return: $732.95
Total Trades: 75
Average Profit per Trade: $9.77


In [23]:
import vectorbt as vbt

xly = xly_copy.dropna(subset=['Close_XLY'])
entries = xly_copy['Buy_Signal'].astype(bool)
exits = xly_copy['Sell_Signal'].astype(bool)

price = xly_copy['Close_XLY']
portfolio = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001
)

print(portfolio.stats())
portfolio.plot().show()



ModuleNotFoundError: No module named 'vectorbt'

In [70]:
import vectorbt as vbt

# Make sure index is datetime and 'Close_TSLA' exists
price = merged_data['Close_XLY']

# Generate entries and exits from your signals
entries = merged_data['Buy_Signal']
exits = merged_data['Sell_Signal']

# Create portfolio
portfolio = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    size=np.nan,  # Let it auto-calculate position size if fixed capital
    init_cash=100_000,
    fees=0.001,  # 0.1% per trade
    slippage=0.0005  # Optional
)

# Plot portfolio value
portfolio.plot().show()


In [71]:
# Summary stats
print(portfolio.stats())

# Equity curve
portfolio.plot().show()

# You can compare to SPY, TSLA Buy-and-Hold too


Start                              0.000000
End                               19.000000
Period                            20.000000
Start Value                   100000.000000
End Value                     100000.000000
Total Return [%]                   0.000000
Benchmark Return [%]             -10.163929
Max Gross Exposure [%]             0.000000
Total Fees Paid                    0.000000
Max Drawdown [%]                        NaN
Max Drawdown Duration                   NaN
Total Trades                       0.000000
Total Closed Trades                0.000000
Total Open Trades                  0.000000
Open Trade PnL                     0.000000
Win Rate [%]                            NaN
Best Trade [%]                          NaN
Worst Trade [%]                         NaN
Avg Winning Trade [%]                   NaN
Avg Losing Trade [%]                    NaN
Avg Winning Trade Duration              NaN
Avg Losing Trade Duration               NaN
Profit Factor                   

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



In [72]:
xly = merged_data.dropna(subset=['Close_XLY'])
entries = merged_data['Buy_Signal'].astype(bool)
exits = merged_data['Sell_Signal'].astype(bool)


In [73]:
price = merged_data['Close_XLY']
portfolio = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001
)

print(portfolio.stats())
portfolio.plot().show()



/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                              0.000000
End                               19.000000
Period                            20.000000
Start Value                   100000.000000
End Value                      90286.704981
Total Return [%]                  -9.713295
Benchmark Return [%]             -10.163929
Max Gross Exposure [%]           100.000000
Total Fees Paid                  296.373903
Max Drawdown [%]                   9.980462
Max Drawdown Duration             17.000000
Total Trades                       2.000000
Total Closed Trades                1.000000
Total Open Trades                  1.000000
Open Trade PnL                 -7950.098224
Win Rate [%]                       0.000000
Best Trade [%]                    -1.764960
Worst Trade [%]                   -1.764960
Avg Winning Trade [%]                   NaN
Avg Losing Trade [%]              -1.764960
Avg Winning Trade Duration              NaN
Avg Losing Trade Duration          2.000000
Profit Factor                   